## Subgrounds Data Query

### Sushi Olympus Liquidity Pools:

#### V2
* OHM-DAI - 0x055475920a8c93cffb64d039a8205f7acc7722d3 
* OHM-WETH - 0x69b81152c5a8d35a67b32a4d3772795d96cae4da
* OHM-BTRFLY - 0xe9ab8038ee6dd4fcc7612997fe28d4e22019c4b4
* OHM-FRAX - 0x04fc84d0b2914005cd16fa377ae889d768b2e7ff
* OHM-LUSD - 0x46e4d8a1322b9448905225e52f914094dbd6dddf
#### V1
* OHM-DAI - 0x34d7d7aaf50ad4944b70b320acb24c95fa2def7c
* OHM-WETH - 0xfffae4a0f4ac251f4705717cd24cadccc9f33e06
* OHM-BTRFLY - 0x96f8c74707c544f654e02e098bb83f69640241b6
* OHM-FRAX - 0x6c765d6b957dacac398d0996cc3e32bd599c8f79
* OHM-LUSD - 0xfdf12d1f85b5082877a6e070524f50f6c84faa6b

#### 9/20/22
* Olympus v2 OIP Sushi to Balancer LP Migration Date Cutoff - https://forum.olympusdao.finance/d/1234-oip-103-balancer-migration
* Olympus v1 -> Olympus V2 Migration (majority migrated December 18th) - https://etherscan.io/tx/0xf4a9cccf2885f1f81e14b31070969b05bcaaeb13531946db97358acee117b35a
* Olympus V3 Framework - https://twitter.com/_ndigo/status/1561397936466149376


In [1]:
import pandas as pd

from subgrounds.subgrounds import Subgrounds

In [2]:
sg = Subgrounds()

subgraph_endpoint = 'https://api.thegraph.com/subgraphs/name/sushiswap/exchange'

sushi = sg.load_subgraph(subgraph_endpoint)

In [3]:
from subgrounds.subgraph import SyntheticField


# datetime synthetic field
sushi.Swap.datetime = SyntheticField.datetime_of_timestamp(sushi.Swap.timestamp)

# fee accrual synthetic field
sushi.Swap.feeRevenue =  sushi.Swap.amountUSD * .0025

In [4]:
# ohm v2 pools
ohmv2_dai = sushi.Query.pair(id='0x055475920a8c93cffb64d039a8205f7acc7722d3')
ohmv2_weth = sushi.Query.pair(id='0x69b81152c5a8d35a67b32a4d3772795d96cae4da')
ohmv2_btrfly = sushi.Query.pair(id='0xe9ab8038ee6dd4fcc7612997fe28d4e22019c4b4')
ohmv2_frax = sushi.Query.pair(id='0x04fc84d0b2914005cd16fa377ae889d768b2e7ff')
ohmv2_lusd = sushi.Query.pair(id='0x46e4d8a1322b9448905225e52f914094dbd6dddf') #Note: OHM is in token 1, LUSD is in token 0. Need to switch them around

# ohm v1 pools?
ohmv1_dai = sushi.Query.pair(id='0x34d7d7aaf50ad4944b70b320acb24c95fa2def7c')
ohmv1_weth = sushi.Query.pair(id='0xfffae4a0f4ac251f4705717cd24cadccc9f33e06')
ohmv1_btrfly = sushi.Query.pair(id='0x96f8c74707c544f654e02e098bb83f69640241b6')
ohmv1_frax = sushi.Query.pair(id='0x6c765d6b957dacac398d0996cc3e32bd599c8f79')
ohmv1_lusd = sushi.Query.pair(id='0xfdf12d1f85b5082877a6e070524f50f6c84faa6b')

In [5]:
ohmv2_lp_ids = [
    '0x055475920a8c93cffb64d039a8205f7acc7722d3',
    '0x69b81152c5a8d35a67b32a4d3772795d96cae4da',
    '0xe9ab8038ee6dd4fcc7612997fe28d4e22019c4b4',
    '0x04fc84d0b2914005cd16fa377ae889d768b2e7ff',
    '0x46e4d8a1322b9448905225e52f914094dbd6dddf'
]

ohmv1_lp_ids = [
    '0x34d7d7aaf50ad4944b70b320acb24c95fa2def7c',
    '0xfffae4a0f4ac251f4705717cd24cadccc9f33e06',
    '0x96f8c74707c544f654e02e098bb83f69640241b6',
    '0x6c765d6b957dacac398d0996cc3e32bd599c8f79',
    '0xfdf12d1f85b5082877a6e070524f50f6c84faa6b'
]

In [ ]:
def get_swaps(data, pools: list, size: int) -> pd.DataFrame:
    '''Wrapper function around subgrounds query function customized for sushi LP pools
    '''
    lp_swap_data = data.swaps(
        orderBy=sushi.Swap.timestamp,
        orderDirection='desc',
        first=size,
        where= {'pair_in': pools}
    )
    df = sg.query_df([
        lp_swap_data.datetime,
        lp_swap_data.timestamp,
        lp_swap_data.transaction.id,
        lp_swap_data.sender,
        lp_swap_data.to,
        lp_swap_data.amount0In,
        lp_swap_data.amount0Out,
        lp_swap_data.amount1In,
        lp_swap_data.amount1Out,
        lp_swap_data.amountUSD,
        lp_swap_data.feeRevenue,
        lp_swap_data.pair.id,
        lp_swap_data.pair.token0.symbol,
        lp_swap_data.pair.token1.symbol
    ])
    return df

In [13]:
df

,swaps_datetime,swaps_timestamp,swaps_transaction_id,swaps_sender,swaps_to,swaps_amount0In,swaps_amount0Out,swaps_amount1In,swaps_amount1Out,swaps_amountUSD,swaps_feeRevenue,swaps_pair_id,swaps_pair_token0_symbol,swaps_pair_token1_symbol
0,2022-09-27 02:49:11,1664261351,0xdccdb79ce6db29247fedd04d849306d077e6f65aea02...,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0.000000,332.491147,2.625756,0.000000,3647.081506,9.117704,0x69b81152c5a8d35a67b32a4d3772795d96cae4da,OHM,WETH
1,2022-09-27 02:49:11,1664261351,0xdccdb79ce6db29247fedd04d849306d077e6f65aea02...,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0.000000,1108.617803,12128.840540,0.000000,12130.216213,30.325541,0x055475920a8c93cffb64d039a8205f7acc7722d3,OHM,DAI
2,2022-09-27 02:45:11,1664261111,0xa5621ee224cd5f339aed026b99ed1da94d1300032bf2...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0x41aeb3e7ce5caca5e734eceacfd82012b7d9334c,0.000000,3.736796,40.065236,0.000000,40.511689,0.101279,0x04fc84d0b2914005cd16fa377ae889d768b2e7ff,OHM,FRAX
3,2022-09-27 02:20:59,1664259659,0x407c0462ea6f806c6b980f51af80dc0aca7d4c8f219f...,0x0eae044f00b0af300500f090ea00027097d03000,0x0eae044f00b0af300500f090ea00027097d03000,160.696553,0.000000,0.000000,1.259875,1751.293980,4.378235,0x69b81152c5a8d35a67b32a4d3772795d96cae4da,OHM,WETH
4,2022-09-27 02:19:11,1664259551,0xdf8df90cbf61b88db0ceca09ffe9ee6a1fe40b21f8ea...,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0.000000,387.877148,4225.928147,0.000000,4225.888629,10.564722,0x055475920a8c93cffb64d039a8205f7acc7722d3,OHM,DAI
5,2022-09-27 02:09:23,1664258963,0x3d99d941a7d5399be48135d0ac564e2e83d7f622250e...,0xc6cf51f57969129654c5014748e44e2a8f6ebd81,0x76f4eed9fe41262669d0250b2a97db79712ad855,0.000000,94.607098,1029.364805,0.000000,1031.474581,2.578686,0x055475920a8c93cffb64d039a8205f7acc7722d3,OHM,DAI
6,2022-09-27 01:49:59,1664257799,0x61f63d1c18a20a433c13350612462334884e76b295f3...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0.000000,636.378074,5.001939,0.000000,6903.019486,17.257549,0x69b81152c5a8d35a67b32a4d3772795d96cae4da,OHM,WETH
7,2022-09-27 01:49:35,1664257775,0xcdfe9c14659e9c622b8f2a01569d44fda089a4080840...,0x00000000c2cf7648c169b25ef1c217864bfa38cc,0x00000000c2cf7648c169b25ef1c217864bfa38cc,0.000000,191.991389,1.500000,0.000000,2069.415916,5.173540,0x69b81152c5a8d35a67b32a4d3772795d96cae4da,OHM,WETH
8,2022-09-27 01:49:35,1664257775,0x64b8e565792e7d405c3e6b9a0fd9a360935166fa93d2...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0x397ff1542f962076d0bfe58ea045ffa2d347aca0,466.791664,0.000000,0.000000,3.632359,5012.446510,12.531116,0x69b81152c5a8d35a67b32a4d3772795d96cae4da,OHM,WETH
9,2022-09-27 01:46:59,1664257619,0x4df9e40cfe3ddcebaa00d5df873bc11ed86dc896c72d...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0.000000,460.250817,5000.000000,0.000000,5002.812683,12.507032,0x055475920a8c93cffb64d039a8205f7acc7722d3,OHM,DAI


In [5]:
# ohmv2_lp_list = [ohmv2_dai, ohmv2_weth, ohmv2_btrfly, ohmv2_frax, ohmv2_lusd]
# ohmv1_lp_list = [ohmv1_dai, ohmv1_weth, ohmv1_btrfly, ohmv1_frax, ohmv1_lusd]
# need v1 and v2 lP lists... OHM symbol in v1 is same as OHM symbol v2. Need a custom field to say whether its v1 or v2, probably add that outside of query

In [ ]:
where = {'pair_in': ['0x055475920a8c93cffb64d039a8205f7acc7722d3', '0x34d7d7aaf50ad4944b70b320acb24c95fa2def7c']}

In [6]:
def get_sushi_query(data, size: int) -> pd.DataFrame:
    '''Wrapper function around subgrounds query function customized for sushi LP pools
    '''
    lp_swap_data = data.swaps(
        orderBy=sushi.Swap.timestamp,
        orderDirection='desc',
        first=size
    )
    df = sg.query_df([
        lp_swap_data.datetime,
        lp_swap_data.timestamp,
        lp_swap_data.transaction.id,
        lp_swap_data.sender,
        lp_swap_data.to,
        # lp_swap_data.pair.reserve0,
        # lp_swap_data.pair.reserve1,
        lp_swap_data.amount0In,
        lp_swap_data.amount0Out,
        lp_swap_data.amount1In,
        lp_swap_data.amount1Out,
        lp_swap_data.amountUSD,
        lp_swap_data.feeRevenue,
        # lp_swap_data.pair.id,
        lp_swap_data.pair.token0.symbol,
        lp_swap_data.pair.token1.symbol
    ])
    return df

In [7]:
def get_query_data(lp_list: list, size: int) -> list[pd.DataFrame]:
    '''
    Returns a list of dataframes from list of sushi liquidity pool addresses.
    '''
    data = []

    for pool in lp_list:
    # iterate through every sushi lp
        swap_df = get_sushi_query(pool, size)

    # set index to datetime
        swap_df.set_index('pair_swaps_datetime')

    # If OHM isn't token0, switch column names
        if swap_df['pair_swaps_pair_token0_symbol'][0] != 'OHM':
            # swap column names
            a = swap_df['pair_swaps_pair_token0_symbol'][0]
            print(f'swap_df needs column change: {a}') # debugging statement

            swap_df = swap_df.rename(columns={
                'pair_swaps_sender': 'pair_swaps_to', 
                'pair_swaps_to': 'pair_swaps_sender',
                'pair_swaps_amount0In': 'pair_swaps_amount1In', 
                'pair_swaps_amount0Out': 'pair_swaps_amount1Out',
                'pair_swaps_amount1In': 'pair_swaps_amount0In', 
                'pair_swaps_amount1Out': 'pair_swaps_amount0Out',
                'pair_swaps_pair_token0_symbol': 'pair_swaps_pair_token1_symbol', 
                'pair_swaps_pair_token1_symbol': 'pair_swaps_pair_token0_symbol'

            })

    # append data to swap_df
        data.append(swap_df)
        
    return data



In [8]:
# get list of lp swap dataframe data
ohmv1_data_list = get_query_data(ohmv1_lp_list, 10000000) # only gets data starting in October 2021


# took 58 mins to run

In [ ]:
# ohmv2_data_list = get_query_data(ohmv2_lp_list, 10000000)

In [9]:
# concat merges all of the dataframes together
ohmv1_df = pd.concat(ohmv1_data_list)
# ohmv2_df = pd.concat(ohmv2_data_list)

# set index to datetime
ohmv1_df = ohmv1_df.set_index('pair_swaps_datetime')
# ohmv2_df = ohmv2_df.set_index('pair_swaps_datetime')

In [10]:
# save to csv
ohmv1_df.to_csv('ohmv1_df.csv')
# ohmv2_df.to_csv('ohmv2_df.csv')

### Debugging, making sure dataset is clean

In [12]:
ohmv1_df['pair_swaps_pair_token0_symbol'].unique() #debugging
ohmv1_df['pair_swaps_pair_token0_symbol'].unique() #debugging

ohmv2_df['pair_swaps_pair_token1_symbol'].unique() #debugging
ohmv2_df['pair_swaps_pair_token1_symbol'].unique() #debugging

array(['DAI', 'WETH', 'BTRFLY', 'FRAX', 'LUSD'], dtype=object)

In [13]:
ohmv1_df

,pair_swaps_timestamp,pair_swaps_transaction_id,pair_swaps_sender,pair_swaps_to,pair_swaps_amount0In,pair_swaps_amount0Out,pair_swaps_amount1In,pair_swaps_amount1Out,pair_swaps_amountUSD,pair_swaps_feeRevenue,pair_swaps_pair_token0_symbol,pair_swaps_pair_token1_symbol
pair_swaps_datetime,,,,,,,,,,,,
2022-09-21 03:25:11,1663745111,0x639fa56914c4a1e72dde17e6f5c5461bb5b08090c8d2...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0x055475920a8c93cffb64d039a8205f7acc7722d3,0.180000,0.000000,0.000000,9.169886,9.176697,0.022942,OHM,DAI
2022-09-20 21:39:23,1663724363,0x6aaf7271932300f424d73d5d33cef9591da1b73f11cb...,0xf2f400c138f9fb900576263af0bc7fcde2b1b8a8,0x1111111254fb6c44bac0bed2854e76f90643097d,0.000000,6.092630,307.860077,0.000000,308.117726,0.770294,OHM,DAI
2022-09-20 18:35:23,1663713323,0x6c22b500565f7a888d6e77504e9d31fc76bc53f5d4da...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0xaaf5110db6e744ff70fb339de037b990a20bdace,0.537892,0.000000,0.000000,26.660424,26.666630,0.066667,OHM,DAI
2022-09-20 18:25:47,1663712747,0xfad3b8711575a96b7c74af8d2e7e17b292ced0c4278e...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0xa037d0238a594bac23685fbcc768341cdda95827,0.677373,0.000000,0.000000,33.670688,33.680734,0.084202,OHM,DAI
2022-09-20 18:18:35,1663712315,0xc21dd8b0189534206d4619888a8aa386f1793046f641...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0x8b07a6f6693d2d5c04c850a011f00984e2f70a05,0.020084,0.000000,0.000000,1.000000,0.999703,0.002499,OHM,DAI
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-06 20:17:36,1633565856,0x815c829efcec50f5abd82425b6b105732692c48cb8d2...,0x911605012f87a3017322c81fcb4c90ada7c09116,0x2dce0dda1c2f98e0f171de8333c3c6fe1bbf4877,0.000000,9.035073,6479.773586,0.000000,6786.560446,16.966401,OHM,LUSD
2021-10-06 20:17:05,1633565825,0x1296f7f66d4cf552ddef88b759ce795a6360e87e4b61...,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,0x5abfbe56553a5d794330eaccf556ca1d2a55647c,16.407626,0.000000,0.000000,12056.931315,12468.009260,31.170023,OHM,LUSD
2021-10-06 19:24:35,1633562675,0x17981a29bfcaa31162e7ff1c05c71615346d7ec94613...,0x00000000116579a5ba59e2f22e77ede26809b970,0x279ca79d5fb2490721512c8ae4767e249d75f41b,5.146897,0.000000,0.000000,4111.247357,4123.243931,10.308110,OHM,LUSD


In [15]:
ohmv2_df

,pair_swaps_timestamp,pair_swaps_transaction_id,pair_swaps_sender,pair_swaps_to,pair_swaps_amount0In,pair_swaps_amount0Out,pair_swaps_amount1In,pair_swaps_amount1Out,pair_swaps_amountUSD,pair_swaps_feeRevenue,pair_swaps_pair_token0_symbol,pair_swaps_pair_token1_symbol
pair_swaps_datetime,,,,,,,,,,,,
2022-09-21 07:12:23,1663758743,0x04ff62f3b04d462553276c22689a076b19df5f6b024c...,0xc6cf51f57969129654c5014748e44e2a8f6ebd81,0x76f4eed9fe41262669d0250b2a97db79712ad855,0.000000,109.285916,1076.108666,0.000000,1076.527368,2.691318,OHM,DAI
2022-09-21 06:52:35,1663757555,0xc73d03a6721ea4cf9148709deb706647902711747772...,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0.000000,128.274253,1262.287859,0.000000,1262.200159,3.155500,OHM,DAI
2022-09-21 06:24:23,1663755863,0x5bb90d675d3c51c3ebcf7c653f7ce5d81a88155a0751...,0x1111111254fb6c44bac0bed2854e76f90643097d,0xf931e52ba14843c80cedb13b9ac906ce8d61f8fc,0.000000,29.011443,285.369601,0.000000,285.417742,0.713544,OHM,DAI
2022-09-21 05:53:23,1663754003,0x96958683234796440de1ef83b678bcfcebf7c5533b6d...,0xc6cf51f57969129654c5014748e44e2a8f6ebd81,0x76f4eed9fe41262669d0250b2a97db79712ad855,0.000000,117.116496,1151.565010,0.000000,1152.180684,2.880452,OHM,DAI
2022-09-21 05:33:11,1663752791,0x90869754086eb88dcb2138757637620e05f7dc9ae994...,0xf2f400c138f9fb900576263af0bc7fcde2b1b8a8,0xf2f400c138f9fb900576263af0bc7fcde2b1b8a8,0.000000,407.372915,4000.000000,0.000000,3998.394083,9.995985,OHM,DAI
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14 13:35:07,1647279307,0x31838ba464ed9e45769bb4bde89925aa87a95bc844f3...,0x16980c16811bde2b3358c1ce4341541a4c772ec9,0xa1006d0051a35b0000f961a8000000009ea8d2db,420.873995,0.000000,0.000000,11900.149689,5946.554173,14.866385,OHM,LUSD
2022-03-14 03:07:38,1647241658,0x124e60e2d826b35bc06b33ff1f3dad20726bfe5f4e52...,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,951.627540,0.000000,0.000000,27000.192142,13565.788789,33.914472,OHM,LUSD
2022-03-14 00:06:19,1647230779,0x1d8cd8bf8e73387afcc677c8df930c353cd65ec4efd6...,0xe9c9f537ea2d07768421c142a10e08ce26932e70,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,40.983596,0.000000,0.000000,1165.716381,586.076045,1.465190,OHM,LUSD
